In [18]:
from executorch.extension.llm.runner import MultimodalRunner, GenerationConfig, make_image_input, make_text_input
from executorch.kernels import quantized
from transformers import AutoProcessor
from executorch.extension.llm.custom_ops import custom_ops

In [6]:
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)
image_url = "https://llava-vl.github.io/static/images/view.jpg"
conversation = [
    {
        "role": "system", 
        "content": [
            {
                "type": "text", 
                "text": "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions."
            }]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": image_url},
            {
                "type": "text",
                "text": "What are the things I should be cautious about when I visit here?",
            },
        ],
    },
]
inputs = processor.apply_chat_template(conversation, add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt")


Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 20815.40it/s]


In [7]:
print(processor.apply_chat_template(conversation))

A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: <image>
What are the things I should be cautious about when I visit here? 


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained("/Volumes/larryliu/work/models/llava/")

('/Volumes/larryliu/work/models/llava/tokenizer_config.json',
 '/Volumes/larryliu/work/models/llava/special_tokens_map.json',
 '/Volumes/larryliu/work/models/llava/chat_template.jinja',
 '/Volumes/larryliu/work/models/llava/tokenizer.model',
 '/Volumes/larryliu/work/models/llava/added_tokens.json',
 '/Volumes/larryliu/work/models/llava/tokenizer.json')

In [ ]:
inputs_combined = [
    make_text_input("A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions. USER: "), 
    make_image_input(inputs["pixel_values"]), 
    make_text_input("\nWhat are the things I should be cautious about when I visit here? "),
]
runner = MultimodalRunner("/Volumes/larryliu/work/models/llava/model.pte", "/Volumes/larryliu/work/models/llava/tokenizer.model", None)
config = GenerationConfig()
config.max_new_tokens = 100
runner.generate(inputs_combined, config)

E tokenizers:hf_tokenizer.cpp:60] Error parsing json file: [json.exception.parse_error.101] parse error at line 2, column 1: syntax error while parsing value - invalid literal; last read: '<U+000A><U+000E>'
E tokenizers:tiktoken.cpp:59] invalid tiktoken line: 
[llm_runner_helper.cpp:77] Loaded Sentencepiece tokenizer
[llm_runner_helper.cpp:270] Reading metadata from model
[llm_runner_helper.cpp:133] Metadata: use_sdpa_with_kv_cache = 1
[llm_runner_helper.cpp:133] Metadata: use_kv_cache = 1
[llm_runner_helper.cpp:131] Method get_max_context_len not found, using the default value 128
[llm_runner_helper.cpp:133] Metadata: get_max_context_len = 128
[llm_runner_helper.cpp:133] Metadata: get_max_seq_len = 2048
[llm_runner_helper.cpp:131] Method enable_dynamic_shape not found, using the default value 0
[llm_runner_helper.cpp:133] Metadata: enable_dynamic_shape = 0
[llm_runner_helper.cpp:144] Setting kMaxContextLen to kMaxSeqLen value: 2048
[multimodal_runner.cpp:88] RSS after loading model: 0


What are the things I should be cautious about when I visit here? ASSISTANT: 1

[multimodal_runner.cpp:132] RSS after multimodal input processing: 0.000000 MiB (0 if unsupported)
[multimodal_runner.cpp:144] Max new tokens resolved: 100, pos_ 634, max_context_len 2048


. Weather conditions: Since the image shows a dock surrounded by a large body of water, it is important to be aware of the weather conditions. Unfavorable weather, such as strong winds, heavy rain, or storms, can make the dock slippery and increase the risk of accidents.

2. Tides and water levels: Be aware of the tides and water levels, as they can affect the dock's stability and safety. Tides can


[stats.h:108] 	Prompt Tokens: 634    Generated Tokens: 99
[stats.h:114] 	Model Load Time:		8.312000 (seconds)
[stats.h:124] 	Total inference time:		57.655000 (seconds)		 Rate: 	1.717110 (tokens/second)
[stats.h:132] 		Prompt evaluation:	30.963000 (seconds)		 Rate: 	20.476052 (tokens/second)
[stats.h:143] 		Generated 99 tokens:	26.692000 (seconds)		 Rate: 	3.708976 (tokens/second)
[stats.h:151] 	Time to first generated token:	30.963000 (seconds)
[stats.h:158] 	Sampling time over 733 tokens:	0.006000 (seconds)


PyTorchObserver {"prompt_tokens":634,"generated_tokens":99,"model_load_start_ms":1758244266102,"model_load_end_ms":1758244274414,"inference_start_ms":1758244274414,"inference_end_ms":1758244332069,"prompt_eval_end_ms":1758244305377,"first_token_ms":1758244305377,"aggregate_sampling_time_ms":6,"SCALING_FACTOR_UNITS_PER_SECOND":1000}


In [45]:
from executorch.extension.pybindings.portable_lib import _load_for_executorch

In [46]:
module = _load_for_executorch("/Volumes/larryliu/work/models/llava/model.pte")

In [26]:
print(module.method_meta("text_decoder"))

MethodMeta(name='text_decoder', num_inputs=2, input_tensor_meta=['TensorInfo(sizes=[1, 2047, 4096], dtype=Float, is_memory_planned=False, nbytes=33538048)', 'TensorInfo(sizes=[2047], dtype=Long, is_memory_planned=False, nbytes=16376)'], num_outputs=1, output_tensor_meta=['TensorInfo(sizes=[1, 2047, 32064], dtype=Float, is_memory_planned=True, nbytes=262540032)'])


In [27]:
image = inputs["pixel_values"]

In [47]:
res = module.run_method("vision_encoder", [image])

In [48]:
print(res[0].shape)

torch.Size([1, 576, 4096])


In [33]:
from transformers import LlavaForConditionalGeneration
model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype="auto", device_map="auto")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.46s/it]


In [34]:
model.config.name_or_path

'llava-hf/llava-1.5-7b-hf'

In [35]:
"llava" in model.config.name_or_path

True

In [37]:
import torch
class VisionExportableModule(torch.nn.Module):
    def __init__(self, model: torch.nn.Module):
        super().__init__()
        self.model = model

    def prepare_export_inputs(self):
        # 1. Get export inputs
        model_id = self.model.config.name_or_path
        processor = AutoProcessor.from_pretrained(model_id)
        sample_conversation_with_image = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "url": "https://llava-vl.github.io/static/images/view.jpg"},
                ],
            },
        ]
        processed_inputs = processor.apply_chat_template(
            sample_conversation_with_image,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        )
        if "pixel_values" not in processed_inputs:
            raise ValueError(
                f"Unable to obtain sample audio encoder inputs for export for {model_id} - the processor did not return formatted inputs with the 'pixel_values' key: {processed_inputs}"
            )
        export_inputs = processed_inputs["pixel_values"]

        # 2. Get export dynamic shapes
        dynamic_shapes = None  # No batching for now.

        return export_inputs, dynamic_shapes

    def forward(
        self,
        input_features: torch.FloatTensor,
    ):
        image_embeds = self.model.get_image_features(input_features)
        # Hack, assuming the text decoder will take a 3D tensor (batch_size, seq_len, hidden_size)
        if "llava" in self.model.config.name_or_path:
            # Llava returns a list of 2D tensors (seq_len, hidden_size)
            return image_embeds[0].unsqueeze(0)
        else:
            return image_embeds

In [41]:
vision = VisionExportableModule(model.to("cpu"))

In [43]:
a = vision.forward(image.to("cpu"))

In [44]:
a.shape

torch.Size([1, 576, 4096])